In [ ]:
// Define the target variable and input dataset. Ensure that the input variables are assigned before executing this script.
// This automation script generates two tables. It first calculates the relevancy (individual variable contribution to the response), followed by the interaction of input variables using Random Forest.
// 1. The importance of all variables will be saved in the table hiveschema.FinalVariableImportance.
// 2. Next, we retrieve the top 250 variables from the first step, build a Random Forest model, and list the variable importances in descending order in the table schema.final_var_importancev.
val target1="target" 
val tablename="hivetablename"
val username="userschema"
val numberofvar="50"   // This parameter limits the number of variables to Random Forest
// sampling  method - oversampling
val Target_1=20000
val Target_0=40000
 
 
 
 
 //don't modify anything below unless needed
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types.StringType
import org.apache.spark.sql.functions._
import scala.collection.mutable.{ArrayBuffer, Map}
import org.apache.spark.SparkContext._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.feature.ChiSqSelector
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types.StringType
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Interaction
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.classification.RandomForestClassificationModel
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.regression.{RandomForestRegressionModel, RandomForestRegressor}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}



//don't modify anything below
 def rankCategory(categoryFrequency: scala.collection.Map[String,Long]) : scala.collection.Map[String,Int] = {

     val frequency =  categoryFrequency.map(_._2).toSeq.distinct.sorted
     val freqIndex = frequency.zipWithIndex.toMap
     val categoryRanked : scala.collection.Map[String,Int] = categoryFrequency.map{ case (category, frequency) =>
       (category,freqIndex.getOrElse(frequency,0))
     }
     categoryRanked
   }

   def convertDataFrameToWordFrequency(inputDF : DataFrame) : DataFrame = {
     var outputDataFrame = inputDF
     val schema =  inputDF.schema
     schema.filter(_.dataType == StringType).foreach( structField => {
       val colName = structField.name
       val numOfCategories  =  outputDataFrame.select(colName).distinct().count()
       if(numOfCategories > 80){
         // delete the column object
         outputDataFrame = outputDataFrame.drop(colName)
       }
       else{
        val categoryFrequency : scala.collection.Map[String,Long] =  outputDataFrame.select(colName).groupBy(colName).agg(count("*").alias("count")).rdd.
           map{case row => (row.getAs[String](colName), row.getAs[Long]("count"))}.collectAsMap()
        val categoryWithRank  : scala.collection.Map[String,Int] = rankCategory(categoryFrequency)
        val convertStringToRankUDF = udf[Int,String]((s:String) =>{
          categoryWithRank.getOrElse(Option(s).getOrElse(""),-1)
        })
         // converts string to their rank(Integer)
         outputDataFrame = outputDataFrame.
           withColumn(colName,convertStringToRankUDF(col(colName)) )
       }
     })
     outputDataFrame
   }

  //val inputDataFrame1 = spark.sql(s"select * from (select  cast( $target1 as int)  as target1,a.* from  $tablename a where  cast( $target1 as int)=1 limit 50000 )a1 union  select * from (select  cast( $target1 as int)  as target1,a.* from  $tablename a where  cast( $target1 as int)=0 limit 100000 )a2") 
  val inputDataFrame1 = spark.sql(s"select  cast( $target1 as int)  as target1,a.* from  $tablename a where  cast( $target1 as int)=1  ")
  val inputDataFrame2 = spark.sql(s"select  cast( $target1 as int)  as target1,a.* from  $tablename a where  cast( $target1 as int)=0  ")
  def getRandom (inputDF : DataFrame, n : Int) : DataFrame = {
    val count = inputDF.count();
    val howManyTake = if (count > n) n else count;
    val outputDataFrame1=inputDF.sample(true,1.0*howManyTake/count).limit (n)
    outputDataFrame1
   }
   val interim1=getRandom(inputDataFrame1,s"$Target_1".toInt)
   val interim2=getRandom(inputDataFrame1,s"$Target_0".toInt)
   interim2.count
   val inputDataFrame1=interim1.union(interim2)
   inputDataFrame1.select(inputDataFrame1.columns.filter(_.contains("id","key","ID","KEY")).map(inputDataFrame1(_)) : _*).drop()
   inputDataFrame1.select(inputDataFrame1.columns.filter(_.startsWith("id")).map(inputDataFrame1(_)) : _*).drop()
   inputDataFrame1.select(inputDataFrame1.columns.filter(_.endsWith("id")).map(inputDataFrame1(_)) : _*).drop()
   
   
      val count1=inputDataFrame1.count
      var inputDataFrame=spark.emptyDataFrame
      if (count1 >= 10000000){
       inputDataFrame = inputDataFrame1.sample(false, 0.02)
      } else if(count1>5000000 & count1 <10000000){
         inputDataFrame=inputDataFrame1.sample(false,0.04)
      } else if(count1 <5000000 & count1>2000000){
         inputDataFrame=inputDataFrame1.sample(false,0.06)
      } else if(count1 <2000000 & count1>500000){
         inputDataFrame=inputDataFrame1.sample(false,0.3)
      } else if(count1 >200000 & count1 <=500000) {
      println("hi")
         inputDataFrame=inputDataFrame1.sample(false,0.8)
         inputDataFrame.cache()
         inputDataFrame.count()
      } else {
       println("hi")
       inputDataFrame=inputDataFrame1
      }
    inputDataFrame.count()
    
      val dataFrameWithCharRank = convertDataFrameToWordFrequency(inputDataFrame)
       dataFrameWithCharRank.show()

       val fields = dataFrameWithCharRank.schema.fields filter {
       x => x.dataType match { 
             case x: org.apache.spark.sql.types.DateType => true
             case x: org.apache.spark.sql.types.TimestampType => true
             case _ => false 
             } 
           } map { x => x.name }

       val newDf = fields.foldLeft(dataFrameWithCharRank){ case(dframe,field) => dframe.drop(field) }
           var newDf1 = newDf.drop("target")
       var output3 = ""
       val a=newDf1.columns.foreach(a => output3+="corr("+a+",target1) as "+a+",")
       newDf1.registerTempTable("variablereduction")
       var b=output3.dropRight(1)
        val varimportance=spark.sql(s"select $b from variablereduction")
          val schema1 =  varimportance.schema
          val outPutList = ArrayBuffer.empty[String] 
          schema1.foreach( structField1 => {
          var outPutLine = ""
          val colName = structField1.name
          //val numOfCategories1  =  varimportance.map{x => (x.getAs[Double]("colName"))}
          val numOfCategories1  =  varimportance.select(colName).collectAsList.get(0).toString
          outPutLine += colName + "|" + numOfCategories1
          outPutList += outPutLine
          })
          
        val hadoopConf=sc.hadoopConfiguration

       var hdfs = org.apache.hadoop.fs.FileSystem.get(hadoopConf)
       hdfs.delete(new org.apache.hadoop.fs.Path(s"/user/$username/varimp"))

       sc.parallelize(List(outPutList.mkString("\n").replaceAll("[\\[\\]]", ""))).repartition(1).saveAsTextFile(s"/user/$username/varimp")
        val textfile1=sc.textFile(s"/user/$username/varimp")
        spark.sql(s"drop table if exists $username.FinalVariableIMportance")  
         textfile1.map(x => {val cols=x.split("\\|", -1)
                            (cols(0),cols(1))}).toDF("variablenames","RankVI").write.saveAsTable(s"$username.FinalVariableIMportance")
        
       val variablefinallist=spark.sql(s"select b.variablenames from (select row_number() over ( order by c.RankVI1 Desc) as rowid,c.* from (select abs(cast(RankVI as decimal(18,12))) as RankVI1 , a.* from $username.FinalVariableIMportance a) c where UPPER(c.variablenames) not in('TARGET','HOUSEKEY','TARGET1') )B where  RankVI<=0.8 and rowid<=$numberofvar").toDF.collect.toArray.map(_.get(0)).map(value => if(value != null) value.toString() else value).map(_.toString)

        
       val nonmissing=newDf1.na.fill(-1)
           val indexer = new StringIndexer().setInputCol("target1").setOutputCol("label")
         val assembler = new VectorAssembler().setInputCols(variablefinallist).setOutputCol("features")
          val output = assembler.transform(nonmissing)

           val labelIndexer = new StringIndexer().setInputCol("target1").setOutputCol("indexedLabel").fit(output)
       val featureIndexer = new VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(output)
        val Array(trainingData, testData) = output.randomSplit(Array(0.7, 0.3))
       val rf = new RandomForestClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setNumTrees(10).setFeatureSubsetStrategy("auto").setImpurity("gini").setMaxDepth(4).setMaxBins(32)

       val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)

        val pipeline = new Pipeline().setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))
        val model = pipeline.fit(trainingData)
        val predictions = model.transform(testData)
         val Tpredictions = model.transform(trainingData)
       predictions.select("predictedLabel", "target1", "features").show(5)
       val evaluator = new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy")
       val precision1 =new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("weightedPrecision").evaluate(predictions)
       val roc_evaluator = new BinaryClassificationEvaluator().setLabelCol("target1").setRawPredictionCol("prediction").setMetricName("areaUnderROC").evaluate(predictions)
       val roc_evaluator1 = new BinaryClassificationEvaluator().setLabelCol("target1").setRawPredictionCol("prediction").setMetricName("areaUnderROC").evaluate(Tpredictions)

       val accuracy = evaluator.evaluate(predictions)
       val accuracy1 = evaluator.evaluate(Tpredictions)
       println("Test Error = " + (1.0 - accuracy))
       println("Training Error = " + (1.0 - accuracy1))
        println("precision ="+ precision1)
         println("precision ="+ roc_evaluator)
           println("precision ="+ roc_evaluator1)
       val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
        println("Learned classification forest model:\n" + rfModel.toDebugString)
        var varimpvalues  = model.stages(2).asInstanceOf[RandomForestClassificationModel].featureImportances.toArray
        var rfModel1  = model.stages(2).asInstanceOf[RandomForestClassificationModel].featureImportances.size
       var indexVar = 0
       val outputlistfinal1 = ArrayBuffer.empty[String] 
        while(indexVar < rfModel1) {
       var outputlist3 = ""
       outputlist3+=variablefinallist(indexVar)+"|"+varimpvalues(indexVar)
       outputlistfinal1+=outputlist3
       indexVar = indexVar + 1
        }
        spark.sql(s"drop table if exists $username.final_var_importance")  
                val hadoopConf=sc.hadoopConfiguration

       var hdfs = org.apache.hadoop.fs.FileSystem.get(hadoopConf)
       hdfs.delete(new org.apache.hadoop.fs.Path(s"/user/$username/final_var_importance"))
        
        
         sc.parallelize(List(outputlistfinal1.mkString("\n"))).repartition(1).saveAsTextFile(s"/user/$username/final_var_importance")
          val textfile2=sc.textFile(s"/user/$username/final_var_importance") 
         textfile2.map(x => {val cols=x.split("\\|", -1)
                            (cols(0),cols(1))}).toDF("variablenames","RandomF_VF").write.mode("overwrite").saveAsTable(s"$username.final_var_importance")

         spark.sql(s"select * from $username.final_var_importance order by  RandomF_VF desc ").show(40,false)